In this notebook we show how to read and interpolate a series of CTD profiles using the `DIVAnd` interpolation tool.

In [2]:
import Pkg; Pkg.add("NCDatasets")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [85f8d34a] + NCDatasets v0.12.7
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using DIVAnd
using NCDatasets
using PyPlot

# User inputs

## Domain of interest, resolution

In [38]:
depthmin, depthmax, deltadepth = 0., 100., 1.;
timemin, timemax, deltatime = 0., 1955., 1.;
depthr = depthmin:deltadepth:depthmax;
timer = timemin:deltatime:timemax;

## Files and directories

In [6]:
datadir = "./data/";
datafile = joinpath(datadir, "Stareso100m_2012_2013_2017_divand.txt")
resultdir = joinpath("./results/divand/");
resultfile = joinpath(resultdir, "Stareso100m_2012_2013_2017.nc");
info("Data file:\n", datafile)
info("Result file:\n", resultfile)

12-mai 16:36:52:INFO:root:Data file:
./data/Stareso100m_2012_2013_2017_divand.txt
12-mai 16:36:52:INFO:root:Result file:
./results/divand/Stareso100m_2012_2013_2017.nc


# Read values from file

Create function to read.

In [7]:
function readCTDFile(fname)

    data = readlines(open(fname,"r"))
    nobs = length(data)

    time = zeros(nobs)
    depth = zeros(nobs)
    field = zeros(nobs)

    for i in 1:nobs
        rec = split(data[i])
        time[i] = parse(Float64,rec[1])
        depth[i] = parse(Float64,rec[2])
        field[i] = parse(Float64,rec[3])
    end
    
    return time, depth, field
end
info("Data read from file ", datafile)

12-mai 16:36:53:INFO:root:Data read from file ./data/Stareso100m_2012_2013_2017_divand.txt


In [20]:
if isfile(datafile)
    debug("File", datafile, " exists")
    time, depth, field = readCTDFile(datafile);
    info("Reading ", length(time), " data points")
    info("Mean field value = ", mean(field))
else
    warn("File", datafile, " doesn't exist")
end

12-mai 16:46:05:DEBUG:root:File./data/Stareso100m_2012_2013_2017_divand.txt exists
12-mai 16:46:05:INFO:root:Reading 17824 data points
12-mai 16:46:05:INFO:root:Mean field value = 16.995158283102917


# Apply `divand` interpolation.

In [39]:
mask, (pt, pd), (ti, di) = divand_rectdom(timer, depthr);

## Analysis parameters

In [43]:
# correlation length
len = (5., 10.);
# obs. error variance normalized by the background error variance
epsilon2 = .5;

## Perform interpolation

In [44]:
@time fi,s = divandrun(mask, (pt, pd), (ti, di), (time, depth), field, len, epsilon2; alphabc=2);

 13.743950 seconds (5.93 M allocations: 2.673 GB, 8.36% gc time)


## Export results to netCDF

In [45]:
sz = size(mask)
dims = [NcDim("depth",sz[1]), NcDim("time",sz[2])];
nc = NetCDF.create(resultfile, NcVar("Temperature", dims))
nc["Temperature"][:,:] = fi
NetCDF.close(nc);
info("Written interpolated field in file:\n", resultfile)

12-mai 16:59:56:INFO:root:Written interpolated field in file:
./results/divand/Stareso100m_2012_2013_2017.nc
